<a href="https://colab.research.google.com/github/deivismartinez/NER-Medical-Uninorte/blob/main/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
#!pip install transformers
!pip install --no-cache-dir transformers sentencepiece
!pip install spacy
!python -m spacy download es_core_news_md
!pip install flair
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-22 17:40:02.527073: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-22 17:40:02.527231: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-22 17:40:02.527261: W tensorflo

In [7]:
from transformers import pipeline, AutoTokenizer
from transformers.models.bert.modeling_bert import BertModel,BertForMaskedLM
import json
from pathlib import Path
from google.colab import files
from os import listdir
from os.path import isfile, join
import spacy
from matplotlib import pyplot as plt
from math import ceil
import shutil
import os
import zipfile
import glob
from flair.data import Sentence
from flair.models import SequenceTagger
import stanza

In [ ]:
tagger = SequenceTagger.load("flair/ner-multi")

In [8]:
stanza.download('es') # download Spanish model
nlp_stanza = stanza.Pipeline('es') # initialize Spanish neural pipeline

INFO:stanza:Downloading default packages for language: es (Spanish) ...
INFO:stanza:File exists: /root/stanza_resources/es/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: es (Spanish):
| Processor    | Package  |
---------------------------
| tokenize     | ancora   |
| mwt          | ancora   |
| pos          | ancora   |
| lemma        | ancora   |
| constituency | combined |
| depparse     | ancora   |
| sentiment    | tass2020 |
| ner          | conll02  |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
!wget https://zenodo.org/record/4279323/files/meddocan.zip?download=1 -O meddocan.zip

In [ ]:
!unzip meddocan.zip

In [9]:
nlp = spacy.load("es_core_news_md")

In [10]:
dataset = {}
for path in (Path('dev'), Path('test'), Path('train')):
    dir_path = Path('meddocan')/path/Path('brat')
    filenames = tuple(f[:-4] for f in listdir(dir_path) if isfile(join(dir_path, f)) if f[-4:] == '.txt')
    dataset[str(path)] = []
    for file_name in filenames:
      d = dict()
      with open(dir_path/Path(file_name+'.txt'), 'r') as f:
        dataset[str(path)].append({"text":f.read(),"file":file_name,"file_name_path":dir_path/Path(file_name+'.txt')})

In [ ]:
!wget https://raw.githubusercontent.com/deivismartinez/NER-Medical-Uninorte/main/tag.json

In [11]:
def get_models():
  models = [
    {"id":1,"name":"roberta-large-bne-capitel-ner", "folder":"PlanTL-GOB-ES/"},
    {"id":2,"name":"roberta-base-bne-capitel-ner-plus", "folder":"PlanTL-GOB-ES/"},
    {"id":3,"name":"roberta-base-bne-capitel-ner", "folder":"PlanTL-GOB-ES/"},
    {"id":4,"name":"bsc-bio-ehr-es-pharmaconer", "folder":"PlanTL-GOB-ES/"},
    {"id":5,"name":"roberta-base-bne-capitel-pos", "folder":"PlanTL-GOB-ES/"},
    {"id":6,"name":"bert-base-multilingual-cased-ner-hrl", "folder":"Davlan/"},
    {"id":7,"name":"xlm-roberta-base-wikiann-ner", "folder":"Davlan/"},
    {"id":8,"name":"xlm-roberta-base-ner-hrl", "folder":"Davlan/"},
    {"id":9,"name":"distilbert-base-multilingual-cased-ner-hrl", "folder":"Davlan/"},
    {"id":10,"name":"xlm-roberta-large-ner-hrl", "folder":"Davlan/"},
    {"id":11,"name":"bert-spanish-cased-finetuned-ner", "folder":"mrm8488/"},
    {"id":12,"name":"TinyBERT-spanish-uncased-finetuned-ner", "folder":"mrm8488/"},
    {"id":13,"name":"RuPERTa-base-finetuned-ner", "folder":"mrm8488/"},
    {"id":14,"name":"ner-spanish-large", "folder":"flair/"},
    {"id":15,"name":"ner-multi", "folder":"flair/"},
    {"id":16,"name":"anglicisms-spanish-mbert", "folder":"lirondos/"},
    {"id":17,"name":"anglicisms-spanish-flair-cs", "folder":"lirondos/"},
    {"id":18,"name":"xlm-roberta-large-ner-spanish", "folder":"MMG/"},
    {"id":19,"name":"wikineural-multilingual-ner", "folder":"Babelscape/"},
    {"id":20,"name":"distilbert-base-multilingual-cased-finetuned-conll2003-ner", "folder":"gunghio/"},
    {"id":21,"name":"bertin-base-ner-conll2002-es", "folder":"bertin-project/"},
    {"id":22,"name":"xlm-roberta-base-finetuned-panx-ner", "folder":"gunghio/"},
    {"id":23,"name":"stanza-es", "folder":"stanfordnlp/"},
    ]
  return models

In [12]:
def select_model(model_index):
  models = get_models()
  if model_index > len(models):
    #print("Model index not exist, from 1 to "+str(len(models)))
    return None
  else:
    model = models[model_index-1].get("folder") + models[model_index-1].get("name")
    return model

In [13]:
def save_models(folder_base):
  models = get_models()
  models_dict = {"models":models}
  with open(folder_base+"models.json", "w") as fp:
    json.dump(models_dict, fp)

In [ ]:
save_models('system')

In [14]:
def get_ner_pipe(model):
  ner_pipe = pipeline(task="ner", model = model)
  return ner_pipe

In [15]:
file = open("tag.json","r")
tag = json.load(file)
file.close()
def get_tag(entity):
  tags=["LOC","PER","ORG", "OTH"]
  for tag_l in tags:
    if tag_l in entity:
      return tag[tag_l]
  return "NEW TAG"

In [16]:
def get_tokenizer(model):
  tokenizer = AutoTokenizer.from_pretrained(model,add_prefix_space=True)
  #do_grafic(tokenizer)
  return tokenizer

In [17]:
def do_grafic(tokenizer):
  token_length = [len(tokenizer(x.get('text'))['input_ids']) for x in dataset['test']]
  plt.hist(token_length)

In [18]:
def change_start_end(list_values, n_string):
  #print(f"list_values:::::::: {list_values}")
  for entity in list_values:
    entity["start"] = entity.get("start") + n_string
    entity["end"] = entity.get("end") + n_string
  return list_values

In [19]:
def build_file_new(entity_list, file_name, min_score = 0.5):
  t = 0
  #print(f"LISTA:::: {entity_list}")
  file= open(file_name,"w")
  end_last = -1
  entity_last = ""
  word_last = ""
  #print(f"LISTA:::: {entity_list}")
  for entity in entity_list:
      #print('--------------Entity Actual::: ',entity['entity'],'-----------',entity['word'].replace('Ġ',' ').replace('Ã±','ñ').replace('Ã¡','á').replace('Ã©','é').replace('ÃŃ','í').replace('Ã³','ó').replace('Ãº','ú'))
      if entity['score'] > min_score:
        if (entity['start'] == end_last or end_last == -1) or ((entity['start'] == (end_last+1)) 
        and get_tag(entity['entity']) == get_tag(entity_last)):
          word_last += entity['word']
          if end_last == -1:
            start_first = entity['start']
            entity_first = entity['entity']
        else:
          t += 1
          end_now = entity['end']
          entity_now = get_tag(entity_first)
          row = 'T'+str(t) +'\t'+ str(entity_now) +' ' + str(start_first) +' '+ str(end_last) +'\t' + word_last.replace('Ġ',' ').replace('Ã±','ñ').replace('Ã¡','á').replace('Ã©','é').replace('ÃŃ','í').replace('Ã³','ó').replace('Ãº','ú') +'\n'
          if(entity_now!='OTHER'):
            file.write(row)
          word_last = entity['word']
          start_first = entity['start']
          entity_first = entity['entity']
        end_last = entity['end']
        entity_last = entity['entity']
  t += 1
  entity_now = get_tag(entity_first)
  row = 'T'+str(t) +'\t'+ str(entity_now) +' ' + str(start_first) +' '+ str(end_last) +'\t' + word_last.replace('Ġ',' ').replace('Ã±','ñ').replace('Ã¡','á').replace('Ã©','é').replace('ÃŃ','í').replace('Ã³','ó').replace('Ãº','ú') +'\n'
  #print(f"ROW::::::::::: {row}")
  #print(f"entity_now::::::::::: {entity_now}")
  #print(f"file_name::::::::::: {file_name}")
  if(entity_now!='OTHER'):
    file.write(row)
  file.close ()

In [20]:
def get_values_512(texto, ner_pipe, model_index, tokenizer):
  max_length = 479
  
  n_string = -1
  ner_output = []
  model = select_model(model_index)
  if model != None:
    #tokenizer = get_tokenizer(model)
    tokens = tokenizer(texto)['input_ids']
    #print(f"Cantidad de tokens totals = {len(tokens)}")
    if len(tokens) % max_length == 0:
      max_length = 477
    if len(tokens) > max_length:
      n_partitions = ceil(len(tokens) / max_length)
      for i in range(n_partitions):
        #print(f"ESte es el valor de i = {i}")
        #print(f"ESte es el valor de n_partitions = {n_partitions}")
        end_index = (i+1)*max_length
        #print(end_index)
        if end_index > len(tokens):
          #print(f"SI ENTRA AL ULTIMO  -   -----  {n_partitions}")
          end_index  = len(tokens) - 1
        #print(end_index)
        token_sequence = tokens[i*max_length:end_index]
        #print(token_sequence)
        string_sequence = tokenizer.decode(token_sequence, 
                                          skip_special_tokens=True,
                                          clean_up_tokenization_spaces=False)
        #print(f"string_sequence = {string_sequence}")
        new_list = change_start_end(ner_pipe(string_sequence), n_string)
        #print(f"new_list = {new_list}")
        ner_output.extend(new_list)
        n_string = n_string + len(string_sequence)
      if n_string != len(texto):
        print(f'Original length: {len(texto)} vs Decoded Size {n_string}')
    else:
      ner_output.extend(ner_pipe(texto))
  if len(ner_output) < 1:
    #ner_output.append({'entity': 'B-ENG', 'score': 0.9959002, 'index': 165, 'word': 'e', 'start': 0, 'end': 0})
    print(texto)
  return ner_output

In [21]:
def get_values_estanza(text):
  list_int = []
  i = 0
  doc = nlp_stanza(text)
  for entity in doc.entities:
    i+=1
    dictonary = {}
    dictonary["entity"] = entity.type
    dictonary["score"] = 0.7
    dictonary["index"] = i
    dictonary["word"] = entity.text
    dictonary["start"] = entity.start_char
    dictonary["end"] = entity.end_char
    list_int.append(dictonary)
  return list_int

In [22]:
def get_values_flair(text):
  sentence = Sentence(text)
  tagger.predict(sentence)
  entity = sentence.get_spans('ner')
  list = []
  i = 0
  for one_entity in entity:
    i+=1
    dictonary = {}
    dictonary["entity"] = one_entity.get_label().value
    dictonary["score"] = round(one_entity.get_label().score,4)
    dictonary["index"] = i
    dictonary["word"] = one_entity.text
    dictonary["start"] = one_entity.start_position
    dictonary["end"] = one_entity.end_position
    list.append(dictonary)
  return list

In [23]:
def get_values_sents(nlp_var, ner_pipe):
  entity_list =  []
  sum = 0
  #print(type(nlp_var))
  for text_npl in nlp_var.sents:
    for entity in ner_pipe(str(text_npl)):
      entity["start"] = entity.get("start") + sum
      entity["end"] = entity.get("end") + sum
      entity_list.append(entity)
    sum = sum + len(str(text_npl))
  return entity_list

In [24]:
def get_values_split(text, ner_pipe):
  entity_list =  []
  text_split = text.split('.')
  sum = 0
  for text_npl in text_split:
    #print(text_npl)
    for entity in ner_pipe(str(text_npl)):
      entity["start"] = entity.get("start") + sum
      entity["end"] = entity.get("end") + sum
      entity_list.append(entity)
    sum = sum + len(str(text_npl))
  return entity_list

In [25]:
def prepare_file(text, file_name, folder_system, min_score,
                 type_text, ner_pipe, model_index, tokenizer):
  file_ann_name = folder_system + file_name + ".ann"
  nlp_var = nlp(text)
  entity_list = []
  if type_text == 0:
    entity_list = ner_pipe(str(nlp_var))
  elif type_text == 1:
    entity_list = get_values_sents(nlp_var, ner_pipe = ner_pipe)
  elif type_text == 2:
    #print(text)
    entity_list = get_values_estanza(text)
    
    #print(model_index)
    #entity_list = get_values_512(text, ner_pipe = ner_pipe,
    #                             model_index=model_index, tokenizer=tokenizer) #para los casos atipicos
    #print(f"LISTA:::: {entity_list}")
  elif type_text == 3:
    entity_list = get_values_split(text, ner_pipe = ner_pipe)
  else:
    print("Type not exist")
  #print(file_name)
  if len(entity_list) > 0:
    build_file_new(entity_list,file_ann_name, min_score = min_score)

In [ ]:
file_did = glob.glob("/content/system/model_23/test2/*.ann")
file_gold = glob.glob("/content/system/gold/*.ann")
print(len(file_gold))
print(len(file_did))

i = 0
nombres = [historia_name[21:-4] for historia_name in file_gold]
nombres_did = [historia_name1[31:-4] for historia_name1 in file_did]

print(f"SOLO NOMBRES = {len(nombres)}")
print(nombres)
print(nombres_did)
faltantes=[]
for name in nombres:
  if name not in nombres_did:
    i=i+1
    faltantes.append(name)
    print(name+ "    -----    " +str(i))

In [39]:
print(faltantes)

[]


In [35]:
def builder(min_score, quantity = 1, type_text = 0, model_index = 1):
  model = select_model(model_index)
  if model != None:
    try:
      ner_pipe = get_ner_pipe(model)
      tokenizer = get_tokenizer(model)
    except:
      #print("No se pudo crear pipeline")
      ner_pipe = "get_ner_pipe(model)"
      tokenizer = "get_tokenizer(model)"
    folder_base = "system/"
    folder_system = folder_base+"model_"+str(model_index)+"/test"+str(type_text)+"/"
    folder_gold = folder_base+"gold/"
    if not Path(folder_system).exists():
      path = Path(folder_system)
      path.mkdir(parents=True)
    if not Path(folder_gold).exists():
      path = Path(folder_gold)
      path.mkdir(parents=True)
    save_models(folder_base)
    for text in dataset['test']:
      if text.get('file') in faltantes:
        try:
          path_completo = str(text.get('file_name_path'))
          path_completo = path_completo[:-4]
          prepare_file(text.get('text'), text.get('file'), folder_system,
                      min_score = min_score, type_text = type_text,
                      ner_pipe = ner_pipe, model_index = model_index, tokenizer = tokenizer)
          shutil.copy(path_completo+".txt", folder_gold+text.get('file')+".txt")
          shutil.copy(path_completo+".ann", folder_gold+text.get('file')+".ann")
          shutil.copy(path_completo+".txt", folder_system+text.get('file')+".txt")
        except:
          print(f"NO PROCESADO   -   ----   --   {text.get('file')}")
      quantity -= 1
      print(quantity)
      # outside if comment
      if quantity < 1:
        break

In [31]:
def builder_all():
  for i in range(0,4):
    for j in range(1,25):
      builder(type_text = i, min_score = 0.0, quantity = 1, model_index = j)

In [32]:
def builder_all_test(start_model = 1, end_model = 1, text = 2, quantity_test = 1):
  for number_model in range(start_model,end_model):
    builder(min_score = 0.0, quantity = quantity_test, type_text = text, model_index = number_model)

In [42]:
builder_all_test(start_model = 23, end_model = 24, text = 2, quantity_test = 250)

5
4
3
2
1
0


In [37]:
print(faltantes)

[]


In [ ]:
comprimir()
downloadFile()

In [ ]:
#Las tablas finales con el de test y desarrollo con train
builder(type_text = 2, min_score = 0.0, quantity = 1, model_index = 2)

In [ ]:
builder(type_text = 1, min_score = 0.0, quantity = 1, model_index = 2)

In [ ]:
builder(type_text = 0, min_score = 0.0, quantity = 1, model_index = 2)

In [ ]:
def comprimir():
  fantasy_zip = zipfile.ZipFile('/content/systemd.zip', 'w')
  for folder, subfolders, filesFolder in os.walk('/content/system'): 
      for file in filesFolder:
          if file.endswith('.ann') or file.endswith('.txt') or file.endswith('.json'):
              fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), '/content/system'), compress_type = zipfile.ZIP_DEFLATED)
  fantasy_zip.close()

In [ ]:
def downloadFile():
  files.download('/content/systemd.zip')

In [ ]:
def remove_file():
  from pathlib import Path
  folder_system = "/content/system/"
  folder = Path(folder_system)
  for file in folder.iterdir():
    try:
      file_name = file.name[0:len(file.name)-4]
      filePath=Path(folder_system+file_name+".ann")
      filePath.unlink()
    except OSError as e:
      print(f"Error:{ e.strerror}")